In [18]:
from PIL import Image
import random
import os

In [19]:
base_raw_image_base_dir = r"data\raw_images\base"
target_raw_image_base_dir = r"data\raw_images\target"
learning_data_base_dir = r"data\learng_data"

base_images = os.listdir(base_raw_image_base_dir)
target_images = os.listdir(target_raw_image_base_dir)

In [20]:
class ImageCoord():
    def __init__(self, x, y, w, h, cls = 0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.x2 = self.x + self.w
        self.y2 = self.y + self.h
        self.cls = cls
        
    def __str__(self):
        return f"{self.x}, {self.y}, {self.w}, {self.h}"

    def get_VOLO_format(self):
        return f"{self.cls} {(self.x + (self.w / 2)) / 1920:.6f} {(self.y + (self.h / 2))/1080:.6f} {self.w/1920:.6f} {self.h/1080:.6f}"

    def is_inside_rect(self, x, y):
        if self.x <= x <= self.x+self.w and self.y <= y <= self.y+self.h:
            return True
        else:
            return False

    def is_overlap(self, coord):
        if self.is_inside_rect(coord.x, coord.y) or \
        self.is_inside_rect(coord.x, coord.y2) or \
        self.is_inside_rect(coord.x2, coord.y) or \
        self.is_inside_rect(coord.x2, coord.y2) or \
        coord.is_inside_rect(self.x, self.y) or \
        coord.is_inside_rect(self.x, self.y2) or \
        coord.is_inside_rect(self.x2, self.y) or \
        coord.is_inside_rect(self.x2, self.y2):
            return True
        else:
            return False
           

In [21]:
def create_labels_txt():
    with open(os.path.join(learning_data_base_dir, "labels.txt"), "w") as f:
        for image in target_images:
            f.write(image.split(".")[0])
            f.write("\n")
            

create_labels_txt()

In [22]:
for loop in range(5):
    for base_image in base_images:
        inserted_image_coord = []
        image_a_path = os.path.join(base_raw_image_base_dir, base_image)
        image_a = Image.open(image_a_path)
        a_width, a_height = image_a.size

        
        for i, target_image in enumerate(target_images):
            image_b_path = os.path.join(target_raw_image_base_dir, target_image)
            image_b = Image.open(image_b_path)
            b_width, b_height = image_b.size
    
            x_max = a_width - b_width
            y_max = a_height - b_height
            
            if x_max < 0 or y_max < 0:
                raise ValueError("B.png가 A.png보다 커서 삽입할 수 없습니다!")
    
            cnt = 0
            fail_cnt = 0
            fail_cnt_threshold = 10
            target_cnt = random.randint(0, 10)
            while cnt <target_cnt:
                # 랜덤 좌표 생성
                x1 = random.randint(0, x_max)
                y1 = random.randint(0, y_max)
                x2 = x1+b_width
                y2 = y1+b_height
                temp_coord = ImageCoord(x1, y1, b_width, b_height, i)
            
                check_result = False
                for coord in inserted_image_coord:
                    if coord.is_overlap(temp_coord):
                        check_result = True
                        fail_cnt += 1
                        break

                if fail_cnt >= fail_cnt_threshold:
                    del temp_coord
                    break
            
                if check_result:
                    del temp_coord
                    continue
                
                # B 이미지를 A 이미지 위에 붙이기
                cnt += 1
                image_a.paste(image_b, (x1, y1), image_b.convert("RGBA"))
                inserted_image_coord.append(temp_coord)

        out_image_file_name = f"{loop}_{base_image.split(".")[0]}"
        # 결과 이미지 저장
        output_path = os.path.join(learning_data_base_dir, out_image_file_name + ".jpg")
        image_a.save(output_path)
    
        # 좌표, 클래스 저장    
        annotation_file_path = os.path.join(learning_data_base_dir, out_image_file_name + ".txt")
        with open(annotation_file_path, 'w') as f:
            for coord in inserted_image_coord:
                # 좌표 기록
                f.write(coord.get_VOLO_format())
                f.write("\n")

In [ ]:
# # 두 이미지 파일 경로
# image_a_path = "Screenshot028.png"
# image_b_path = "PerfectRuby.png"

# # 이미지 열기
# image_a = Image.open(image_a_path)
# image_b = Image.open(image_b_path)

# # A의 크기와 B의 크기 가져오기
# a_width, a_height = image_a.size
# b_width, b_height = image_b.size

# # B를 A 내 랜덤 위치에 삽입 가능한 범위 계산
# x_max = a_width - b_width
# y_max = a_height - b_height

# if x_max < 0 or y_max < 0:
#     raise ValueError("B.png가 A.png보다 커서 삽입할 수 없습니다!")


In [15]:
cnt = 0
class ImageCoord():
    def __init__(self, x, y, w, h, cls = 0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.cls = cls
        
    def __str__(self):
        return f"{self.x}, {self.y}, {self.w}, {self.h}"

    def get_VOLO_format(self):
        return f"{self.cls} {(self.x + (self.w / 2)) / 1920:.6f} {(self.y + (self.h / 2))/1080:.6f} {self.w/1920:.6f} {self.h/1080:.6f}"

    def is_inside_rect(self, x, y):
        if self.x <= x <= self.x+self.w and self.y <= y <= self.y+self.h:
            return True
        else:
            return False
        
inserted_image_coord = []

while cnt <100:
    # 랜덤 좌표 생성
    x1 = random.randint(0, x_max)
    y1 = random.randint(0, y_max)
    x2 = x1+b_width
    y2 = y1+b_height

    check_result = False
    for coord in inserted_image_coord:
        if coord.is_inside_rect(x1,y1) or coord.is_inside_rect(x1,y2) or coord.is_inside_rect(x2,y1) or coord.is_inside_rect(x2,y2):
            check_result = True
            break

    if check_result:
        continue
    
    # B 이미지를 A 이미지 위에 붙이기
    image_a.paste(image_b, (x1, y1), image_b.convert("RGBA"))
    inserted_image_coord.append(ImageCoord(x1,y1,b_width, b_height, 0))
    
    cnt += 1
    # 결과 이미지 저장
output_path = "result.png"
image_a.save(output_path)

with open("result.txt", 'w') as f:
    for coord in inserted_image_coord:
        # 좌표 기록
        f.write(coord.get_VOLO_format())
        f.write("\n")